## Практическое задание

  1. Постройте нейронную сеть(берем простую линейную сеть, которую разбирали на уроке: меняем число слоев, число нейронов , типы активации, тип оптимизатора)  на датасет from sklearn.datasets import load_boston. 
  2. Измените функцию потерь и метрику для этой задачи. Постройте 10-15 вариантов и сведите результаты их работы в таблицу  Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

  3. Поработайте с документацией TensorFlow 2. Найти 2-3 полезные команды TensorFlow, не разобранные на уроке (полезные для Вас).

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets import boston_housing 
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import numpy as np

(train_data,train_labels),(test_data,test_labels)=boston_housing.load_data()

In [2]:
# Нормализация.
scaler = StandardScaler()
scaler.fit_transform(train_data)
scaler.transform(test_data)

array([[ 1.55369355, -0.48361547,  1.0283258 , ...,  0.78447637,
        -3.48459553,  2.25092074],
       [-0.39242675, -0.48361547, -0.16087773, ..., -0.30759583,
         0.42733126,  0.47880119],
       [-0.39982927, -0.48361547, -0.86940196, ...,  0.78447637,
         0.44807713, -0.41415936],
       ...,
       [-0.20709507, -0.48361547,  1.24588095, ..., -1.71818909,
         0.37051949, -1.49344089],
       [-0.36698601, -0.48361547, -0.72093526, ..., -0.48960787,
         0.39275481, -0.41829982],
       [-0.0889679 , -0.48361547,  1.24588095, ..., -1.71818909,
        -1.21946544, -0.40449827]])

In [3]:
# Попробуем разные варианты

epochs = 500

epoch = np.arange(epochs+1)

mae_train = {}
mae_test = {}

for layers in range(2, 5):
    for i in [64, 128, 256]:
        for activation in ['relu', 'tanh', 'sigmoid']:

            # создаем рабочую модель model 
            model = Sequential(name='model')
            for k in range(2, layers+1):
                model.add(Dense(  i, activation=activation))

            model.add(Dense(1))
            
            for opt in ['adam', 'rmsprop', 'nadam']:

                # компилируем model 
                model.compile(
                    optimizer=opt,
                    loss='mse',
                    metrics=['mae'],
                    )

                # проводим обучение модели 
                hh_train = model.fit(
                              train_data,
                              train_labels,
                              epochs=epochs,
                              batch_size=128, verbose = 0
                              )
                # вычисляем ошибку для model
                hh_test = model.evaluate(
                              test_data,
                              test_labels
                              )

                mae_train[str(k) + '_' + str(i) + '_' + activation + '_' + opt] = round(hh_train.history['mae'][-1], 3)
                mae_test[str(k) + '_' + str(i) + '_' + activation + '_' + opt] = round(hh_test[1], 3)

4/4 [==============================] - 0s 1ms/step - loss: 25.0729 - mae: 3.0331


In [4]:
print(min(mae_train, key=mae_train.get))
print(min(mae_train.values()))

4_256_relu_nadam
0.998


In [5]:
print(min(mae_test, key=mae_test.get))
print(min(mae_test.values()))

4_256_sigmoid_nadam
3.033


Для данной задачи регрессии лучше себя проявила сигмоида, оптимизатор - надам

In [13]:
model = Sequential(name='model')
model.add(Dense(  256, activation='sigmoid'))
model.add(Dense(  256, activation='sigmoid'))
model.add(Dense(  256, activation='sigmoid'))
model.add(Dense(1))
            
model.compile(
            optimizer='nadam',
            loss='mse',
            metrics=['mae']
            )

                
hh_train = model.fit(
              train_data,
              train_labels,
              epochs=1500,
              batch_size=128, verbose = 0
              )

y_pred = model.predict(test_data)

In [7]:
from sklearn.metrics import mean_absolute_error, r2_score

In [14]:
mean_absolute_error(test_labels, y_pred)

3.3150420862085683

In [15]:
r2_score(test_labels, y_pred)

0.6482449894553013

Изменяя функцию активации, оптимизатор, количество слоев и нейронов в них я пытался найти наиболее оптимальное сочетание для данной задачи, к сожадению, результаты работы сети оставляют желать лучшего, так как датасет слишком мал для нормального обучения.